In [ ]:
# Cell 1: Setup — Imports and Load Necessary Models/Files
import os
import pickle
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# Paths
VECTOR_STORE_DIR = "../vector_store"
FAISS_INDEX_PATH = os.path.join(VECTOR_STORE_DIR, "faiss_index.bin")
METADATA_CSV_PATH = "../data/chunked_metadata.csv"


# Load embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load FAISS index
index = faiss.read_index(FAISS_INDEX_PATH)

# Load chunk metadata
metadata_df = pd.read_csv(METADATA_CSV_PATH)

# Load text generation pipeline (change model_name as per your choice)
generator = pipeline("text-generation", model="gpt2", max_length=256, do_sample=True)


Device set to use cpu


In [36]:
# Cell 2: Semantic Search Retriever

def semantic_search(query, top_k=5):
    """
    Perform semantic search to retrieve top_k relevant chunks based on query.
    """
    query_vector = embedding_model.encode([query])
    distances, indices = index.search(query_vector, top_k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx == -1 or idx >= len(metadata_df):
            continue
        row = metadata_df.iloc[idx]
        results.append({
            "id": row.get("id", ""),
            "product": row.get("product", ""),
            "chunk_index": row.get("chunk_index", ""),
            "chunk_text": row.get("chunk_text", ""),
            "score": dist
        })

    return results


In [42]:
def build_prompt(context_chunks, question, max_chars=1000):
    """
    Builds a prompt using a truncated version of the context to fit the LLM's input size.
    """
    context_text = ""
    total_chars = 0

    for chunk in context_chunks:
        if total_chars + len(chunk) > max_chars:
            break
        context_text += chunk + "\n\n"
        total_chars += len(chunk)

    prompt = (
        "You are a financial analyst assistant for CrediTrust.\n"
        "Your task is to answer questions about customer complaints.\n"
        "Use the following retrieved complaint excerpts to formulate your answer.\n"
        "If the context doesn't contain the answer, say: \"I don't have enough information.\"\n\n"
        f"Context:\n{context_text}\n"
        f"Question:\n{question}\n\n"
        "Answer:"
    )
    return prompt


In [43]:
# Cell 4: Generate Answer from Prompt

def generate_answer(prompt):
    """
    Generate an answer from the prompt using the LLM text generation pipeline.
    """
    # Generate full output including prompt
    full_output = generator(prompt, max_new_tokens=150, do_sample=True)[0]['generated_text']

    # Extract only the answer by removing the prompt part
    answer = full_output[len(prompt):].strip()

    return answer


In [44]:
# Cell 5: Define the retriever function

def retrieve_relevant_chunks(question, top_k=5):
    """
    Embeds the input question and retrieves top_k most relevant text chunks using FAISS.
    """
    try:
        query_vector = model.encode([question])
        distances, indices = index.search(query_vector, top_k)

        retrieved = []
        for dist, idx in zip(distances[0], indices[0]):
            if idx == -1 or idx >= len(metadata_df):
                continue
            row = metadata_df.iloc[idx]
            retrieved.append({
                "id": row["id"],
                "product": row["product"],
                "chunk_index": row["chunk_index"],
                "chunk_text": row["chunk_text"],
                "score": dist
            })

        return retrieved
    except Exception as e:
        print(f"Error retrieving chunks: {e}")
        return []


In [45]:
# Cell 6: Evaluate the RAG Pipeline

# Sample evaluation questions
sample_questions = [
    "What are common issues with credit card fraud?",
    "How do customers describe their problems with savings accounts?",
    "Are there complaints about Buy Now, Pay Later services?",
    "What complaints exist regarding personal loans?",
    "What issues do customers face with money transfers?"
]

# Store evaluation results
eval_results = []

for question in sample_questions:
    retrieved_chunks = retrieve_relevant_chunks(question, top_k=5)
    
    # Skip if no results retrieved
    if not retrieved_chunks:
        eval_results.append({
            "Question": question,
            "Generated Answer": "No relevant context found.",
            "Retrieved Sources": [],
            "Quality Score": None,
            "Comments": "No chunks retrieved."
        })
        continue

    # Extract chunk texts
    context_chunks = [r["chunk_text"] for r in retrieved_chunks]
    
    # Build prompt
    prompt = build_prompt(context_chunks, question)
    
    # Generate answer
    answer = generate_answer(prompt)
    
    # Include top 2 retrieved sources for context in eval
    top_sources = context_chunks[:2]

    eval_results.append({
        "Question": question,
        "Generated Answer": answer,
        "Retrieved Sources": top_sources,
        "Quality Score": None,  # To be filled manually
        "Comments": ""
    })

# Convert results to DataFrame
import pandas as pd
eval_df = pd.DataFrame(eval_results)

# Display preview
eval_df.head()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


,Question,Generated Answer,Retrieved Sources,Quality Score,Comments
0,What are common issues with credit card fraud?,"You may be surprised by this question, ""What i...",[subject deceptive billing practices improper ...,None,
1,How do customers describe their problems with ...,"""I don't know what to do about that. I don't h...",[ally bank has had consistent issues with acce...,None,
2,"Are there complaints about Buy Now, Pay Later ...",All new users are not able to view the purchas...,[ongoing payment situation with best buy my pa...,None,
3,What complaints exist regarding personal loans?,The following are examples of customer complai...,[in xxxxxxxx i took out a 60000 installment lo...,None,
4,What issues do customers face with money trans...,i have a small amount of money with my account...,[i own a xxxx xxxx xxxx xxxx xxxx xxxx a custo...,None,


In [49]:
import pandas as pd
columns = ["Question", "Generated Answer", "Retrieved Sources", "Quality Score", "Comments"]
eval_df = pd.DataFrame(columns=columns)
eval_df.to_csv("rag_evaluation_results.csv", index=False)
print("Empty evaluation CSV file created: rag_evaluation_results.csv")



Empty evaluation CSV file created: rag_evaluation_results.csv
